In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import artm

In [2]:
#read the file containing a lemmatized dataset in csv format
df_docs = pd.read_csv('Amazon.csv', header=None, names=['docs'])
docs = df_docs['docs'].tolist()

In [3]:
#calculate the number of unique words in the dataset
tokens_unique = np.unique(np.concatenate(df_docs['docs'].str.split()))
tokens_unique.size

28422

In [4]:
#specify the number of topics for each level
level1_topics = [2,3,4,5,6,7,8,9,10,11,12,13,14,15, 16, 17, 18, 19, 20,21, 22,23,24, 25,26,27,28,29, 30,31,32,33,34,35,36,37,38,39, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200]
level2_topics = [1]

In [5]:
# max_features should be equal to the number of unique words 
cv = CountVectorizer(max_features=tokens_unique.size, token_pattern=r'\w+', stop_words=None)
n_wd = np.array(cv.fit_transform(docs).todense()).T
vocabulary = cv.get_feature_names()

In [ ]:
#transform the data into a special format for hARTM model
bv = artm.BatchVectorizer(
    data_format='bow_n_wd',
    n_wd=n_wd,
    vocabulary=vocabulary
)

In [ ]:
import time
start = time.time()
myfile = 'hARTM_Renyi_Amazon.csv'

with open(myfile, 'a') as file: 
    file.write('Num topics (lev 1)' + ';' +  'WRD (lev 1)' + ';' + 'prob_sum (lev 1)bv' + ';' + 
               'Renyi(lev 1)' +';' + 'Num topics (lev 2)' + ';' +'WRD (lev 2)' + ';' + 'prob_sum (lev 2)' + ';' + 
             'Renyi(lev 2)' +';' + '\n')
    

for ilevel1 in level1_topics:
    for ilevel2 in level2_topics:
        levels = []
        #run hARTM model
        model = artm.hARTM(num_processors=5, cache_theta=True, dictionary=bv.dictionary, seed = 36, scores = PerplexityScore)
        
        level = model.add_level(num_topics=ilevel1, parent_level_weight=1)
        levels.append(level)
        
        level = model.add_level(num_topics=ilevel2, parent_level_weight=1)
        levels.append(level)
        
        # num_collection_passes is the number of iterations over the whole given collection
        model.fit_offline(bv, num_collection_passes=200)
        
        #calculate Renyi entropy on the first level
        myprob1 = pd.DataFrame(model.get_level(0).get_phi())
        num_word, num_col = myprob1.shape
        
        thresh = 1/num_word
         
        #---------------------------------------------------------------------------------------------------------------
        #select the maximum probability for each word
        mymax = myprob1.max(axis=1)
        #mymax contains the list of maximum probabilities 
        #select the probabilities larger than thresh
        mywrd = mymax[mymax>thresh]
        #calculate the number of such values
        word_ratio1 = len(mywrd)
        #calculate the sum of those probabilities
        sum_prob1 = sum(mywrd)
        #---------------------------------------------------------------------------------------------------------------
        #Shannon entropy
        Sh1  = np.log(word_ratio1/(num_word*ilevel1))
        #Internal energy
        En1 = -np.log(sum_prob1/ilevel1)
        #Free energy
        Fen1 = En1 - Sh1*ilevel1
        if ilevel1==1: 
            Renyi1 = Fen1/(ilevel1)
        else: Renyi1 = Fen1/(ilevel1-1)
                     
        #calculate Renyi entropy on the second level       
        myprob2 = pd.DataFrame(model.get_level(1).get_phi())
        
        num_word, num_col = myprob2.shape
        
        
        #------------------------------------------------------------------------------------------------------------
        #select the maximum probability for each word
        mymax2 = myprob2.max(axis=1)
        # mymax contains the list of maximum probabilities 
        #select the probabilities larger than thresh
        mywrd2 = mymax2[mymax2>thresh]
        # calculate the number of such values
        word_ratio2 = len(mywrd2)
        #calculate the sum of those probabilities
        sum_prob2 = sum(mywrd2)
        #---------------------------------------------------------------------------------------------------------------        
        
       
        #Shannon entropy 
        Sh2  = np.log(word_ratio2/(num_word*ilevel2))
        #Internal energy
        En2 = -np.log(sum_prob2/ilevel2)
        #Free energy
        Fen2 = En2 - Sh2*ilevel2
        #Renyi entropy
        Renyi2 = Fen2/(ilevel2-1)    
        
            
        with open(myfile, 'a') as file: 
                    file.write(str(ilevel1) + ';' + str(word_ratio1) + ';' + str(sum_prob1) + ';'    
                                     + str(Renyi1) +';' + str(ilevel2) + ';' 
                                     + str(word_ratio1) + ';' + str(sum_prob1) + ';'
                                 + str(Renyi2) + ';' + '\n')
                            
                                                                                                                       

        
stop = time.time()
print('time of execution (sec)', stop - start)
print('---------------------------')

        

/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/artm/hierarchy_utils.py:258: UserWarning: Adding level with num_topics = 1 less or equal than parent level's num_topics = 2
  (num_topics, self._levels[-1].num_topics))


topics level_1:  2   WRD:  3618   SumP:  1.35148419102552   Renyi_1:  5.900651995498711


/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/artm/hierarchy_utils.py:258: UserWarning: Adding level with num_topics = 1 less or equal than parent level's num_topics = 3
  (num_topics, self._levels[-1].num_topics))


topics level_1:  3   WRD:  4360   SumP:  1.744257866008411   Renyi_1:  4.731044367349021


/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/artm/hierarchy_utils.py:258: UserWarning: Adding level with num_topics = 1 less or equal than parent level's num_topics = 4
  (num_topics, self._levels[-1].num_topics))


...

/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/artm/hierarchy_utils.py:258: UserWarning: Adding level with num_topics = 1 less or equal than parent level's num_topics = 70
  (num_topics, self._levels[-1].num_topics))


topics level_1:  70   WRD:  28140   SumP:  62.73485329145842   Renyi_1:  4.3217359854794815


/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/artm/hierarchy_utils.py:258: UserWarning: Adding level with num_topics = 1 less or equal than parent level's num_topics = 80
  (num_topics, self._levels[-1].num_topics))


topics level_1:  80   WRD:  28332   SumP:  72.58170292036812   Renyi_1:  4.441903248052693


/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/home/vignatenko/.conda/envs/veraenv/lib/python3.7/site-packages/artm/hierarchy_utils.py:258: UserWarning: Adding level with num_topics = 1 less or equal than parent level's num_topics = 90
  (num_topics, self._levels[-1].num_topics))
